## Script to add display notes to FHIR Questionnaires

- python version 3.7

- upload example as python dict from file
- create a list of python objects
- add new display and extensions
- write to ig folder as json with text elements
- write to FHIR server as json without text elements

In [ ]:
from json import load, dumps
from flatten_json import flatten
import pandas as pd
from csv import writer
import openpyxl
from IPython import display as D
from fhirclient.models import questionnaire as Q
from fhirclient.models import questionnaireresponse as QR
from fhirclient.models import patient as P
from fhirclient.models import address as A
from fhirclient.models import fhirdate as FD
from fhirclient.models import extension as X
from fhirclient.models import coding as C
from fhirclient.models import fhirreference as FR
from requests import get, post, put

#### Open Questionnaires as python model

In [ ]:
in_path = '/Users/ehaas/Documents/FHIR/Argo-Questionnaire/source/examples/'

in_file = [
           'questionnaire-example-audit-c',
           'questionnaire-example-dast',
           'questionnaire-example-housing',
           'questionnaire-example-phq9',
           'questionnaire-example-sampler'
          ]


def get_r(name): # get files
    with open(f'{in_path}{name}.json') as f:
        r = load(f)
        return(Q.Questionnaire(r))
   

### get file and add to list for processing

In [ ]:
q_list=[]
for f in in_file:
    print(f'==============={f}:================')
    q_list.append(get_r(f))
  

### Instantiate designNote extension

In [ ]:
 
hidden = 'http://hl7.org/fhir/StructureDefinition/questionnaire-hidden'
display_note = 'this is hidden text/should not be displayed to the user'
designNote = 'http://hl7.org/fhir/StructureDefinition/designNote'
displayCategory = 'http://hl7.org/fhir/StructureDefinition/questionnaire-displayCategory'

x = X.Extension(dict(
                    url = designNote,
                    valueMarkdown = display_note,
                )
               )


In [ ]:
def add_x(item):
     for e in item.extension:
            if e.url == hidden:
                print(f'=============== add design note to {item.linkId}================')
                item.extension.append(x)
                break


###  add extension to Q

- loop through files
- loop through items to find hidden text
- add extension to the item

In [ ]:
for q in q_list:
    print(q.id)
    for i in q.item:
        
        try:
            add_x(i)
        except TypeError:
             print('No hidden items')
        try:
            for j in i.item:
                try:
                    add_x(j)
                except TypeError:
                     print('No hidden items')
                try:
                    for k in j.item:
                        try:
                            add_x(k)
                        except TypeError:
                             print('No hidden items')
                except TypeError:
                     print('No sub items')
        except TypeError:
            print('No sub items')


### Strip required for display items

In [ ]:
for q in q_list:
    print(q.id)
    for i in q.item:
        if i.required and i.type == 'display':
            print(i.linkId,irequired,i.type)
            i.required = None
        try:
            for j in i.item:
                if j.required and j.type == 'display':
                    print(j.linkId,j.required,j.type)
                    j.required = None
                try:
                    for k in i.item:
                        if k.required and k.type == 'display':
                            print(k.linkId,k.required,j.type)
                            k.required = None
                except TypeError: 
                     print('No sub items')
        except TypeError:
            
             print('No sub items')

### Pretty print output

In [ ]:
for q in q_list:
    print(dumps(q.as_json(),indent =4))

### Write to file

In [62]:
out_path = '/Users/ehaas/Documents/FHIR/Argo-Questionnaire/source/examples/'
#out_path = ''

def write_file(path, name, data): # write file
    with open(f'{path}{name}.json', 'w') as f:
        f.write(data)
        
for q in q_list:
    write_file(path=out_path,name = f'{q.id}',data = dumps(q.as_json(), indent = 4))



### Validate the Resource
Using the $validate operation, the example is validated by a FHIR Reference Server. The results are displayed below in the human readable text as xhtml.



In [ ]:
val_server = 'http://fhirtest.uhn.ca/baseDstu3'
fhir_test_server = 'http://sqlonfhir-stu3.azurewebsites.net/fhir'

headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}

profile = 'http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/qr'


# *********************** validate Resource ********************************
def validate(pyfhir):
    params = {
    'profile': profile
    }
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(
        f'{val_server}/{pyfhir.resource_type}/$validate',
        params = params,
        headers = headers,
        data = dumps(pyfhir.as_json())
        )
    # print(r.status_code)
    # view  output
    return r.json()["text"]["div"]

display(D.HTML(f'<h1>Validation output</h1>{validate(qr)}'))    

TODO:

- run in rendering-tool
- change to timelimit to timeLimit
- not part of QR - edit this out
- udpate comments change impossible to difficult